# Schedule an Online Evaluation with Tracing and AI Inference SDK (deprecated in Azure AI Foundry project)
- You can configure the RecurrenceTrigger based on the class definition here. The code below demonstrates how to configure the RecurrenceTrigger to run the evaluation every 24 hours. You can also configure the trigger to run at a different interval, or at a specific time of day. Check the Trace menu in the Azure AI Foundry to see the results of the evaluation.
- reference: https://learn.microsoft.com/en-us/azure/ai-studio/how-to/online-evaluation

> ✨ ***Note*** <br>
> You application insight need to be created in Azure AI Foundry to trace your generative AI application. <br>
> Prior to setting up online evaluation, ensure you have first set up [tracing for your generative AI application using Azure AI Inference SDK](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/trace-local-sdk?tabs=python).

In [ ]:
import pandas as pd
import os
import json

from pprint import pprint
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import RelevanceEvaluator
from azure.ai.evaluation import GroundednessEvaluator, GroundednessProEvaluator
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    Evaluation,
    Dataset,
    EvaluatorConfiguration,
    ConnectionType,
    EvaluationSchedule,
    RecurrenceTrigger,
    ApplicationInsightsConfiguration
)
import pathlib

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    ContentSafetyEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
    F1ScoreEvaluator,
    RetrievalEvaluator
)

from azure.ai.ml import MLClient



load_dotenv(override=True)

In [ ]:
credential = DefaultAzureCredential()

azure_ai_project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ.get("AZURE_AI_PROJECT_CONN_STR"),  # At the moment, it should be in the format "<Region>.api.azureml.ms;<AzureSubscriptionId>;<ResourceGroup>;<HubName>" Ex: eastus2.api.azureml.ms;xxxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxxxxx;rg-sample;sample-project-eastus2
)

model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
    "type": "azure_openai",
}

In [ ]:
application_insights_connection_string = azure_ai_project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your Azure AI Foundry project page.")
    exit()

from azure.core.settings import settings 
from azure.monitor.opentelemetry import configure_azure_monitor

# https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/trace-local-sdk?tabs=python
os.environ['AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED'] = 'true'
settings.tracing_implementation = "opentelemetry" 

configure_azure_monitor(connection_string=application_insights_connection_string)

In [ ]:
from azure.ai.inference.tracing import AIInferenceInstrumentor 

# Instrument AI Inference API to enable trace instrumentation for AI Inference
AIInferenceInstrumentor().instrument() 

In [ ]:
from openai import AzureOpenAI
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage
from azure.ai.inference.models import UserMessage
from azure.core.credentials import AzureKeyCredential

aoai_inference_endpoint = os.getenv("AZURE_AI_INFERENCE_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

try:
    azure_ai_inference_client = ChatCompletionsClient(
        endpoint = aoai_inference_endpoint,
        credential = AzureKeyCredential(aoai_api_key),
    )

    print("=== Initialized azure_ai_inference_client ===")
    print(f"AZURE_AI_INFERENCE_ENDPOINT={aoai_inference_endpoint}")
    print(f"AZURE_OPENAI_DEPLOYMENT_NAME={aoai_deployment_name}")
        
except (ValueError, TypeError) as e:
    print(e)

In [ ]:
import os
from azure.ai.inference.models import SystemMessage
from azure.ai.inference.models import UserMessage

topic = f"""
Tell me about the history of Microsoft. 
"""

system_message = """
Generate plain text sentences of #topic# related text to improve the recognition of domain-specific words and phrases. using json format. 

"""

user_message = f"""
#topic#: {topic}
"""

# Simple API Call
response = azure_ai_inference_client.complete(
    model=aoai_deployment_name,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ],
    temperature=0.8,
    top_p=0.1
)

content = response.choices[0].message.content
print(content)
print("Usage Information:")
#print(f"Cached Tokens: {response.usage.prompt_tokens_details.cached_tokens}") #only o1 models support this
print(f"Completion Tokens: {response.usage.completion_tokens}")
print(f"Prompt Tokens: {response.usage.prompt_tokens}")
print(f"Total Tokens: {response.usage.total_tokens}")

### Set up online evaluation schedule 

Evaluations are only supported in the same regions as AI-assisted risk and safety metrics. Please find the reference document here. (https://learn.microsoft.com/en-us/azure/ai-studio/how-to/online-evaluation?tabs=linux) 

### Prerequisites

- A new User-assigned Managed Identity in the same resource group and region. Make a note of the clientId; you'll need it later.
- An Azure AI Hub in the same resource group and region.
- An Azure AI project in this hub, see Create a project in Azure AI Foundry portal.
- An Azure Monitor Application Insights resource created in Azure AI Foundry portal.
- Navigate to the hub page in Azure portal and add Application Insights resource, see Update Azure Application Insights
- Azure OpenAI Deployment with GPT model supporting chat completion, for example gpt-4.
- Navigate to your Application Insights resource in the Azure portal and use the Access control (IAM) tab to add the Log Analytics Contributor role to the User-assigned Managed Identity you created previously.
- [Attach the User-assigned Managed Identity to your project.](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-identity-based-service-authentication?view=azureml-api-2&tabs=cli#add-a-user-assigned-managed-identity-to-a-workspace-in-addition-to-a-system-assigned-identity)
- modify managed_identity_sample.yaml to attach the managed identity to your workspace
- az ml workspace update --resource-group <RESOURCE_GROUP> --name <WORKSPACE_NAME> --file <YAML_FILE_NAME>.yaml
- Before attaching the User-assigned Managed Identity to your project, you also need to set the permission on the user assigned managed identity including keyvault access policies. - https://learn.microsoft.com/en-us/azure/machine-learning/how-to-identity-based-service-authentication?view=azureml-api-2&tabs=cli#user-assigned-managed-identity
- If you run into issues attaching the User-assigned Managed Identity to your project, you need to assign Key Vault Contributor role to the workspace (project) resource. 
- Navigate to your Azure AI Services in the Azure portal and use the Access control (IAM) tab to add the Cognitive Services OpenAI Contributor role to the User-assigned Managed Identity you created previously.

In [ ]:
relevance_evaluator_config = EvaluatorConfiguration(
    id=RelevanceEvaluator.id,
    init_params={"model_config": model_config},
    data_mapping={"query": "${data.Input}", "response": "${data.Output}"}
)


# CoherenceEvaluator
coherence_evaluator_config = EvaluatorConfiguration(
    id=CoherenceEvaluator.id,
    init_params={"model_config": model_config},
    data_mapping={"query": "${data.Input}", "response": "${data.Output}"}
)

evaluators_cloud = {
    "relevance": relevance_evaluator_config,
    "coherence" : coherence_evaluator_config
}

In [ ]:
kusto_query = 'let gen_ai_spans=(dependencies | where isnotnull(customDimensions["gen_ai.system"]) | extend response_id = tostring(customDimensions["gen_ai.response.id"]) | project id, operation_Id, operation_ParentId, timestamp, response_id); let gen_ai_events=(traces | where message in ("gen_ai.choice", "gen_ai.user.message", "gen_ai.system.message") or tostring(customDimensions["event.name"]) in ("gen_ai.choice", "gen_ai.user.message", "gen_ai.system.message") | project id= operation_ParentId, operation_Id, operation_ParentId, user_input = iff(message == "gen_ai.user.message" or tostring(customDimensions["event.name"]) == "gen_ai.user.message", parse_json(iff(message == "gen_ai.user.message", tostring(customDimensions["gen_ai.event.content"]), message)).content, ""), system = iff(message == "gen_ai.system.message" or tostring(customDimensions["event.name"]) == "gen_ai.system.message", parse_json(iff(message == "gen_ai.system.message", tostring(customDimensions["gen_ai.event.content"]), message)).content, ""), llm_response = iff(message == "gen_ai.choice", parse_json(tostring(parse_json(tostring(customDimensions["gen_ai.event.content"])).message)).content, iff(tostring(customDimensions["event.name"]) == "gen_ai.choice", parse_json(parse_json(message).message).content, "")) | summarize operation_ParentId = any(operation_ParentId), Input = maxif(user_input, user_input != ""), System = maxif(system, system != ""), Output = maxif(llm_response, llm_response != "") by operation_Id, id); gen_ai_spans | join kind=inner (gen_ai_events) on id, operation_Id | project Input, System, Output, operation_Id, operation_ParentId, gen_ai_response_id = response_id'

# AzureMSIClientId is the clientID of the User-assigned managed identity created during set-up - see documentation for how to find it
properties = {"AzureMSIClientId": os.environ.get("AZURE_MSI_CLIENT_ID")}

service_name = "evaluation_sdk_schedule"

# Your Application Insights resource ID
# At the moment, it should be something in the format "/subscriptions/<AzureSubscriptionId>/resourceGroups/<ResourceGroup>/providers/Microsoft.Insights/components/<ApplicationInsights>""
app_insights_resource_id = os.environ.get("APP_INSIGHTS_RESOURCE_ID")

# Connect to your Application Insights resource
app_insights_config = ApplicationInsightsConfiguration(
    resource_id=app_insights_resource_id, query=kusto_query
)

In [ ]:
# Frequency to run the schedule
recurrence_trigger = RecurrenceTrigger(frequency="Hour", interval=1)

# Configure the online evaluation schedule
evaluation_schedule = EvaluationSchedule(
    data=app_insights_config,
    evaluators=evaluators_cloud,
    trigger=recurrence_trigger,
    description=f"scheduled evaluation",
    properties=properties
)

# Create the online evaluation schedule
created_evaluation_schedule = azure_ai_project_client.evaluations.create_or_replace_schedule(service_name, evaluation_schedule)
print(
    f"Successfully submitted the online evaluation schedule creation request - {created_evaluation_schedule.name}, currently in {created_evaluation_schedule.provisioning_state} state."
)

In [ ]:


evaluation_schedule = azure_ai_project_client.evaluations.get_schedule(service_name)
print(evaluation_schedule)

# Sample for list evaluation schedules
for evaluation_schedule in azure_ai_project_client.evaluations.list_schedule():
    print(evaluation_schedule)

# Sample for disable an evaluation schedule with name
# project_client.evaluations.disable_schedule(service_name)

In [ ]:
count = 0
for evaluation_schedule in azure_ai_project_client.evaluations.list_schedule():
    count += 1
    print(f"{count}.{evaluation_schedule.name} "
    f"[IsEnabled: {evaluation_schedule.is_enabled}]")
    print(f"Total evaluation schedules: {count}")

### go to the workspace to check the instance_results.jsonl files on the Outputs + log tab 

![instance_results](./images/instance_results_jsonl.png)

Disable (soft-delete) online evaluation schedule:

In [ ]:
# target_service_name = "evaluation_sdk_schedule"
# azure_ai_project_client.evaluations.disable_schedule(target_service_name)